In [2]:
# importing the required libraries
import numpy as numpy
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [7]:
#image data generator

train_datagen = ImageDataGenerator(rescale=1./255., preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory("dataset/train",
                                                    target_size=(150,150),
                                                    color_mode="rgb",
                                                    batch_size=16,
                                                    class_mode="categorical")

Found 498 images belonging to 2 classes.


In [8]:
valid_datagen = ImageDataGenerator(rescale=1./255., preprocessing_function=preprocess_input)

valid_generator = valid_datagen.flow_from_directory("dataset/test",
                                                    target_size=(150,150),
                                                    color_mode="rgb",
                                                    batch_size=16,
                                                    class_mode="categorical")

Found 499 images belonging to 2 classes.


## Transfer learning

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [5]:
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(150,150,3))

80142336/80134624 [==============================] - 50s 1us/step


In [6]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [9]:
model_tl = Sequential()
model_tl.add(base_model)
model_tl.add(Flatten())
model_tl.add(Dense(2, activation="softmax"))

In [10]:
model_tl.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 16386     
Total params: 20,040,770
Trainable params: 20,040,770
Non-trainable params: 0
_________________________________________________________________


In [12]:
# compiling
model_tl.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
model_tl.fit(train_generator, validation_data=valid_generator, epochs=10)

Epoch 1/10
32/32 [==============================] - 349s 11s/step - loss: 3.7905 - accuracy: 0.5723 - val_loss: 0.6559 - val_accuracy: 0.5311
Epoch 2/10
32/32 [==============================] - 301s 9s/step - loss: 0.5728 - accuracy: 0.6747 - val_loss: 0.3751 - val_accuracy: 0.8717
Epoch 3/10
32/32 [==============================] - 305s 10s/step - loss: 0.4481 - accuracy: 0.7751 - val_loss: 0.4698 - val_accuracy: 0.7796
Epoch 4/10
32/32 [==============================] - 330s 10s/step - loss: 0.4397 - accuracy: 0.8373 - val_loss: 0.8413 - val_accuracy: 0.5030
Epoch 5/10
32/32 [==============================] - 327s 10s/step - loss: 0.5546 - accuracy: 0.7108 - val_loss: 0.3299 - val_accuracy: 0.8878
Epoch 6/10
32/32 [==============================] - 349s 11s/step - loss: 0.5833 - accuracy: 0.7510 - val_loss: 0.4033 - val_accuracy: 0.8798
Epoch 7/10
32/32 [==============================] - 327s 10s/step - loss: 0.3545 - accuracy: 0.8795 - val_loss: 1.2780 - val_accuracy: 0.5010
Epoch 8